In [1]:
from algorithms.iterative_relaxation import IterativeRelaxation
from algorithms.decision_procedure import MarabouCoreDP
from models.test_models import ProphecyPaperNetwork, TestModel
import torch
import copy
from torch import nn

Instructions for updating:
non-resource variables are not supported in the long term


### Test DP

In [2]:
dp = MarabouCoreDP()
prophecy_model = ProphecyPaperNetwork()

In [3]:
activation = {
  'linear_relu_stack.1': ['ON', 'OFF'], 
  'linear_relu_stack.3': ['ON', 'OFF'],
}
dp.solve(activation, prophecy_model, 0)

['unsat',
 {},

In [4]:
activation = {
  'linear_relu_stack.1': ['ON', 'OFF'], 
  'linear_relu_stack.3': ['--', '--'],
}
dp.solve(activation, prophecy_model, 0)

['unsat',
 {},

In [5]:
activation = {
  'linear_relu_stack.1': ['ON', '--'], 
  'linear_relu_stack.3': ['--', '--'],
}
dp.solve(activation, prophecy_model, 0)

['sat',
 {0: -49.9999995,
  1: -50.0000005,
  2: 1e-06,
  3: -100.0,
  4: 1e-06,
  5: 0.0,
  6: 5e-07,
  7: -5e-07,
  8: 5e-07,
  9: 5e-07,
  10: -0.0,
  11: -0.0},

### Test Iterative Relaxation

In [6]:
ir_model = ProphecyPaperNetwork()
iterative_relaxation = IterativeRelaxation()

input_property = iterative_relaxation.call(ir_model, [[1, -1]], 0)
print(f"input property: {input_property}")

input property: [{'linear_relu_stack.1': ['ON', 'OFF'], 'linear_relu_stack.3': ['--', '--']}]


In [7]:
ir_model = ProphecyPaperNetwork()
iterative_relaxation = IterativeRelaxation()

input_property = iterative_relaxation.call(ir_model, [[1, -1]], 1)
print(f"input property: {input_property}")

input property: [{'linear_relu_stack.1': ['ON', 'OFF'], 'linear_relu_stack.3': ['ON', 'OFF']}, 1]
